# Projeto 3 - Ciência dos Dados

#  Projeto UFC

Nome: Victor Vergara Arcoverde de Albuquerque Cavalcanti

Nome: Edgard Ortiz Neto

Nome: Gabriel Yamashita

Nome: Henrique Mualem Marti



  ___
## Objetivo:

### O objetivo desse projeto é fazer um machine learning para poder prever qual lutador ganhará uma luta do UFC baseado no seu histórico. Para isso serão usados os dados de todas as lutas do UFC (mais de 5 mil) a fim de descobrir quais os fatores dos lutadores que impactam mais no resultado das lutas. 

[Database utilizado](https://www.kaggle.com/rajeevw/ufcdata#data.csv)

____
## Método escolhido:






### Random Forest:
#### Esse método usa várias árvores de decisão para encontrar o que melhor se adequa, a que tem menos erros, para os nossos dados e o resultado que queremos, nesse caso qual é o vencedor.

![randomforest.png](randomforest.png)



### Regressão Logística:
#### Esse método usa a função abaixo que vai sempre tender a 0 ou a 1, assim sendo um classificador binário. Ele atribui um coeficiente(β) para cada fator levado em consideração, assim tendo uma ordem de impacto dos fatores no resultado final.

$$Prob(y = 1 | X = x) = \frac{1}{1 + e^{-\left(\beta_0 + \beta_1 x_1 + \beta_2 x_2\right)}}$$

 ___
## Preparando o ambiente no jupyter:


### Imports:

In [361]:
import math
import os.path
import pandas as pd
import numpy as np
import matplotlib
import matplotlib.pyplot as plt
import json
import random
import statsmodels.api as sm
import seaborn as sns

from sklearn.feature_extraction.text import CountVectorizer
from sklearn.metrics import accuracy_score
from sklearn.naive_bayes import MultinomialNB

from sklearn.pipeline import Pipeline
from sklearn.ensemble import RandomForestClassifier
from sklearn.linear_model import LogisticRegression
from sklearn.model_selection import train_test_split


### Trabalhando com os Excels:

In [362]:
data = pd.read_excel("data.xlsx")
data2 = data
data.head(2)

,R_fighter,B_fighter,Referee,date,location,Winner,title_bout,weight_class,no_of_rounds,B_current_lose_streak,...,R_win_by_KO/TKO,R_win_by_Submission,R_win_by_TKO_Doctor_Stoppage,R_wins,R_Stance,R_Height_cms,R_Reach_cms,R_Weight_lbs,B_age,R_age
0,Henry Cejudo,Marlon Moraes,Marc Goddard,2019,"Chicago, Illinois, USA",Red,True,Bantamweight,5,0,...,2,0,0,8,Orthodox,162.56,162.56,135.0,31.0,32.0
1,Valentina Shevchenko,Jessica Eye,Robert Madrigal,2019,"Chicago, Illinois, USA",Red,True,Women's Flyweight,5,0,...,0,2,0,5,Southpaw,165.10,167.64,125.0,32.0,31.0


In [363]:
data.weight_class = data.weight_class.astype('category')
data.Winner = data.Winner.astype('category')

In [364]:
data.Winner.value_counts()

Red     3470
Blue    1591
Draw      83
Name: Winner, dtype: int64

In [365]:
data_heavy = data.loc[(data.weight_class=='Heavyweight'),:]
bool_to_number = {False: 0, True: 1}
string_to_number = {'Blue': 0, 'Red': 1, 'Draw': 2}
data_heavy['title_bout'] = data_heavy['title_bout'].map(bool_to_number)
data_heavy['Winner'] = data_heavy['Winner'].map(string_to_number)

C:\ProgramData\Anaconda3\lib\site-packages\ipykernel_launcher.py:4: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy
  after removing the cwd from sys.path.
C:\ProgramData\Anaconda3\lib\site-packages\ipykernel_launcher.py:5: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy
  """


#### Blue = 0
#### Red = 1

In [366]:
data_heavy.head(2)

,R_fighter,B_fighter,Referee,date,location,Winner,title_bout,weight_class,no_of_rounds,B_current_lose_streak,...,R_win_by_KO/TKO,R_win_by_Submission,R_win_by_TKO_Doctor_Stoppage,R_wins,R_Stance,R_Height_cms,R_Reach_cms,R_Weight_lbs,B_age,R_age
4,Tai Tuivasa,Blagoy Ivanov,Dan Miragliotta,2019,"Chicago, Illinois, USA",0,0,Heavyweight,3,0,...,2,0,0,3,Southpaw,187.96,190.50,264.0,32.0,26.0
54,Walt Harris,Serghei Spivac,Jerin Valel,2019,"Ottawa, Ontario, Canada",1,0,Heavyweight,3,0,...,4,0,0,4,Southpaw,195.58,195.58,250.0,24.0,35.0


In [367]:
data_heavy_util = data_heavy.drop(['Referee','date','location'], axis=1)
#dados que não se relacionam com os lutadores ou seus resultado

In [368]:
data_heavy_util.head(2)

,R_fighter,B_fighter,Winner,title_bout,weight_class,no_of_rounds,B_current_lose_streak,B_current_win_streak,B_draw,B_avg_BODY_att,...,R_win_by_KO/TKO,R_win_by_Submission,R_win_by_TKO_Doctor_Stoppage,R_wins,R_Stance,R_Height_cms,R_Reach_cms,R_Weight_lbs,B_age,R_age
4,Tai Tuivasa,Blagoy Ivanov,0,0,Heavyweight,3,0,1,0,17.0,...,2,0,0,3,Southpaw,187.96,190.50,264.0,32.0,26.0
54,Walt Harris,Serghei Spivac,1,0,Heavyweight,3,0,0,0,NaN,...,4,0,0,4,Southpaw,195.58,195.58,250.0,24.0,35.0


In [369]:
data_heavy_util.dropna(inplace=True)
data_heavy_util.head(2)

,R_fighter,B_fighter,Winner,title_bout,weight_class,no_of_rounds,B_current_lose_streak,B_current_win_streak,B_draw,B_avg_BODY_att,...,R_win_by_KO/TKO,R_win_by_Submission,R_win_by_TKO_Doctor_Stoppage,R_wins,R_Stance,R_Height_cms,R_Reach_cms,R_Weight_lbs,B_age,R_age
4,Tai Tuivasa,Blagoy Ivanov,0,0,Heavyweight,3,0,1,0,17.0,...,2,0,0,3,Southpaw,187.96,190.5,264.0,32.0,26.0
60,Arjan Bhullar,Juan Adams,1,0,Heavyweight,3,0,1,0,8.0,...,0,0,0,2,Orthodox,185.42,190.5,245.0,27.0,32.0



### Tirando os dados categóricos:

In [370]:
categoricas = [
    'R_fighter', 
    'B_fighter', 
    'weight_class', 
    'R_Stance', 
    'B_Stance', 
]

data_heavy_cat = data_heavy_util[categoricas].astype('category')
data_heavy_num = data_heavy_util.drop(categoricas, axis=1).astype('float')

___
# Teste 1 - Random Forest

In [373]:
X = data_heavy_num.drop('Winner', axis=1)
y = data_heavy_num['Winner']

## Separando os dados em testes e treinamento

In [374]:
X_train_random, X_test_random, y_train_random, y_test_random = train_test_split(X, y, test_size=0.25)


## Montando modelo Random Forest

In [375]:
model_random = RandomForestClassifier(n_estimators=10000)

model_random.fit(X_train_random, y_train_random)

RandomForestClassifier(bootstrap=True, class_weight=None, criterion='gini',
            max_depth=None, max_features='auto', max_leaf_nodes=None,
            min_impurity_decrease=0.0, min_impurity_split=None,
            min_samples_leaf=1, min_samples_split=2,
            min_weight_fraction_leaf=0.0, n_estimators=10000, n_jobs=None,
            oob_score=False, random_state=None, verbose=0,
            warm_start=False)


## Verificando a performance


In [376]:
X_train_random.columns

Index(['title_bout', 'no_of_rounds', 'B_current_lose_streak',
       'B_current_win_streak', 'B_draw', 'B_avg_BODY_att', 'B_avg_BODY_landed',
       'B_avg_CLINCH_att', 'B_avg_CLINCH_landed', 'B_avg_DISTANCE_att',
       ...
       'R_win_by_Decision_Unanimous', 'R_win_by_KO/TKO', 'R_win_by_Submission',
       'R_win_by_TKO_Doctor_Stoppage', 'R_wins', 'R_Height_cms', 'R_Reach_cms',
       'R_Weight_lbs', 'B_age', 'R_age'],
      dtype='object', length=136)

In [377]:
model_random.feature_importances_

array([0.00046581, 0.00122089, 0.00209493, 0.00312977, 0.        ,
       0.0156514 , 0.01430696, 0.00839874, 0.00810652, 0.01050487,
       0.00968605, 0.00876934, 0.00915759, 0.01432088, 0.0120284 ,
       0.00440722, 0.00880816, 0.00843081, 0.00607714, 0.00438213,
       0.01255627, 0.01129933, 0.00725649, 0.00473306, 0.00936382,
       0.00710751, 0.00984999, 0.01132035, 0.00996387, 0.00408235,
       0.00288951, 0.00802656, 0.00817471, 0.00780456, 0.0070561 ,
       0.0091701 , 0.00957419, 0.01114001, 0.01030999, 0.00799547,
       0.00795352, 0.00426553, 0.00767192, 0.00696527, 0.0072793 ,
       0.00377412, 0.00792435, 0.00708076, 0.00895828, 0.00394782,
       0.0069829 , 0.00516577, 0.00713023, 0.00821218, 0.01047267,
       0.00699277, 0.00758641, 0.00192857, 0.00068343, 0.00173646,
       0.0026623 , 0.00444623, 0.00156011, 0.00033796, 0.00413411,
       0.00532367, 0.00987918, 0.01210611, 0.00199588, 0.00384623,
       0.        , 0.00860471, 0.00802705, 0.00983874, 0.00882

In [378]:
y_pred_random = model_random.predict(X_test_random)

In [379]:
print(accuracy_score(y_test_random, y_pred_random))

0.6060606060606061


In [380]:
y_pred_random

array([1., 1., 1., 1., 1., 1., 0., 1., 0., 1., 0., 1., 1., 1., 1., 1., 0.,
       1., 1., 1., 0., 1., 1., 1., 1., 0., 1., 0., 0., 0., 0., 0., 1., 1.,
       1., 1., 1., 1., 0., 0., 1., 1., 1., 1., 1., 1., 1., 1., 0., 1., 1.,
       1., 0., 0., 1., 0., 1., 0., 1., 1., 0., 1., 1., 0., 1., 0.])

In [381]:
y_test_random.value_counts(True)

1.0    0.727273
0.0    0.242424
2.0    0.030303
Name: Winner, dtype: float64

In [382]:
data_heavy_num.Winner.value_counts()

1.0    167
0.0     92
2.0      5
Name: Winner, dtype: int64

   ___
## Conclusão do Modelo inicial Random Forest:

Tendo um acurácia de 0.6667 não é um bom resultado, visto que o modelo praticamente sempre tem como resultado o vermelho como vencedor, e como a probabilidade do lutador vermelho ganhar é de 0.6212112 ele praticamente só acerta os que o vencedor é o vermelho e erra os que o azul é o vencedor.

Assim é possível concluir que é necessário desconsiderar algumas variáveis para melhorar a acurácia.

   ___
## Criando um dataframe dos fatores mais impactantes no resultado segundo o teste acima:

In [383]:
#Fatores que tem o maior peso na decisão da vitória
j=1
lista_j=list()
for i, f in sorted(list(zip(model_random.feature_importances_, X_train_random.columns)), reverse=True):
    a=str(j)+'°'
    lista_j.append(a)
    j+=1

In [384]:
data={'Fator':X_train_random.columns ,'Correlação':model_random.feature_importances_,}
Fator_por_corr=pd.DataFrame(data)
Fator_por_corr=Fator_por_corr.sort_values(by='Correlação', ascending=False)
Fator_por_corr['Grau de Importância']=lista_j
Fator_por_corr = Fator_por_corr.set_index('Grau de Importância')
Fator_por_corr.head(2)

,Fator,Correlação
Grau de Importância,,
1°,R_avg_opp_CLINCH_landed,0.017035
2°,B_avg_BODY_att,0.015651


In [385]:
data_heavy_util_relevante = data_heavy_util.loc[:,uteis]
data_heavy_util_relevante.head(2)

,Winner,B_avg_BODY_att,R_age,B_avg_BODY_landed,B_avg_opp_GROUND_landed,B_avg_SIG_STR_landed,R_avg_opp_KD,R_avg_HEAD_landed,R_avg_opp_GROUND_landed,B_avg_opp_GROUND_att,...,R_avg_SIG_STR_att,R_losses,R_avg_TD_pct,B_avg_opp_TD_att,R_avg_opp_CLINCH_att,B_avg_HEAD_landed,R_avg_BODY_att,B_avg_GROUND_landed,R_avg_PASS,B_avg_opp_TOTAL_STR_landed
4,0,17.0,26.0,14.5,0.0,61.5,0.25,22.75,2.750000,0.0,...,62.250000,1,0.00,0.5,4.5,45.0,7.75,0.0,0.250000,90.0
60,1,8.0,32.0,8.0,2.0,86.0,0.00,24.00,0.666667,2.0,...,50.666667,1,0.72,2.0,8.0,74.0,2.00,35.0,0.666667,46.0


   ___
## Escolhendo quais dados devem ser usados nos modelos de predição:

In [386]:
def relevancia(df,coluna_nome,coluna_correlacao,acuracia):
    inuteis = []
    uteis = ['Winner']
    for index,row in df.iterrows():
        if row[coluna_correlacao] >= -acuracia and row[coluna_correlacao] <= acuracia:
            inuteis.append(row[coluna_nome])
        else:
            uteis.append(row[coluna_nome])
    return uteis

In [387]:
uteis = relevancia(Fator_por_corr,'Fator','Correlação',0.01)

In [388]:
data_heavy_util_relevante = data_heavy_util.loc[:,uteis]
data_heavy_util_relevante.head()

,Winner,R_avg_opp_CLINCH_landed,B_avg_BODY_att,R_avg_opp_CLINCH_att,B_avg_HEAD_att,B_avg_BODY_landed,R_avg_HEAD_landed,R_avg_TD_att,B_avg_SIG_STR_att,R_avg_SIG_STR_att,...,B_avg_opp_GROUND_att,R_avg_opp_KD,R_avg_TOTAL_STR_att,B_avg_DISTANCE_att,B_avg_opp_TOTAL_STR_landed,R_avg_opp_DISTANCE_att,B_avg_opp_GROUND_landed,R_avg_SIG_STR_landed,R_avg_opp_HEAD_landed,R_avg_DISTANCE_att
4,0,3.500000,17.00,4.500000,184.5,14.500,22.75,0.500000,203.50,62.250000,...,0.000,0.25,63.500000,201.000,90.000,42.750000,0.00,32.500000,14.000000,50.750000
60,1,5.333333,8.00,8.000000,131.0,8.000,24.00,2.000000,144.00,50.666667,...,2.000,0.00,98.666667,85.000,46.000,66.666667,2.00,26.666667,12.333333,33.333333
63,1,1.000000,5.00,2.000000,15.5,3.500,21.00,0.000000,22.00,64.000000,...,28.500,0.00,70.000000,16.000,58.000,25.000000,14.50,29.000000,12.000000,51.000000
69,0,3.035714,35.00,4.964286,169.0,21.000,15.25,0.892857,218.00,63.928571,...,0.000,0.25,74.250000,167.000,84.000,40.321429,0.00,26.750000,15.142857,54.178571
75,1,4.200000,4.25,6.466667,28.5,3.875,13.40,0.866667,35.25,40.333333,...,3.625,0.40,51.600000,26.625,31.125,31.333333,2.75,29.466667,13.333333,21.666667


___
# Teste 2- Regressão Logística

____
## Montando a Regressão Logística


In [389]:
def preparo(X,Y):
    X_cp = sm.add_constant(X)
    model = sm.OLS(Y,X_cp,missing='drop')
    results = model.fit()
    return results

In [390]:
Y_log = data_heavy_util_relevante["Winner"]
data_heavy_sem_Winner=data_heavy_util_relevante.drop('Winner',axis=1) 
X_log=data_heavy_sem_Winner
#np.asarray(X)

In [391]:
X_train_log, X_test_log, y_train_log, y_test_log = train_test_split(X_log, Y_log, test_size=0.25)

In [392]:
model = LogisticRegression(max_iter=200000,solver='lbfgs', multi_class='auto')

model.fit(X_train_log, y_train_log)

LogisticRegression(C=1.0, class_weight=None, dual=False, fit_intercept=True,
          intercept_scaling=1, max_iter=200000, multi_class='auto',
          n_jobs=None, penalty='l2', random_state=None, solver='lbfgs',
          tol=0.0001, verbose=0, warm_start=False)

In [393]:
y_pred_log = model.predict(X_test_log)

In [394]:
print(accuracy_score(y_test_log, y_pred_log))

0.6212121212121212


In [395]:
y_pred_log

array([0, 1, 1, 1, 1, 1, 0, 0, 1, 1, 1, 1, 1, 1, 1, 1, 2, 1, 1, 1, 1, 1,
       1, 1, 0, 0, 1, 1, 1, 1, 1, 1, 0, 0, 1, 1, 1, 1, 1, 1, 1, 0, 0, 1,
       0, 1, 1, 1, 0, 1, 1, 2, 1, 1, 1, 1, 1, 1, 1, 1, 0, 1, 0, 1, 1, 0],
      dtype=int64)

In [396]:
y_test_log.value_counts(True)

1    0.575758
0    0.409091
2    0.015152
Name: Winner, dtype: float64

In [397]:
data_heavy_util_relevante.Winner.value_counts()

1    167
0     92
2      5
Name: Winner, dtype: int64

In [398]:
result = preparo(X_log,Y_log)
result.summary()

<class 'statsmodels.iolib.summary.Summary'>
"""
                            OLS Regression Results                            
==============================================================================
Dep. Variable:                 Winner   R-squared:                       0.149
Model:                            OLS   Adj. R-squared:                  0.064
Method:                 Least Squares   F-statistic:                     1.749
Date:                Tue, 19 Nov 2019   Prob (F-statistic):             0.0193
Time:                        15:14:17   Log-Likelihood:                -174.83
No. Observations:                 264   AIC:                             399.7
Df Residuals:                     239   BIC:                             489.1
Df Model:                          24                                         
Covariance Type:            nonrobust                                         
==============================================================================================
                                 coef    std err          t      P>|t|      [0.025      0.975]
----------------------------------------------------------------------------------------------
const                          1.7552      0.509      3.445      0.001       0.752       2.759
R_avg_opp_CLINCH_landed       -0.0105      0.034     -0.309      0.758      -0.077       0.056
B_avg_BODY_att                -0.0236      0.063     -0.376      0.707      -0.147       0.100
R_avg_opp_CLINCH_att           0.0047      0.026      0.183      0.855      -0.046       0.055
B_avg_HEAD_att                -0.0192      0.051     -0.374      0.709      -0.120       0.082
B_avg_BODY_landed              0.0011      0.077      0.014      0.989      -0.151       0.153
R_avg_HEAD_landed              0.0031      0.035      0.090      0.928      -0.065       0.071
R_avg_TD_att                   0.0229      0.026      0.897      0.371      -0.027       0.073
B_avg_SIG_STR_att              0.0209      0.051      0.408      0.684      -0.080       0.122
R_avg_SIG_STR_att             -0.0218      0.028     -0.768      0.443      -0.078       0.034
B_Weight_lbs                  -0.0039      0.002     -1.942      0.053      -0.008    5.73e-05
B_avg_HEAD_landed              0.0180      0.064      0.282      0.778      -0.108       0.144
R_avg_HEAD_att                 0.0186      0.028      0.662      0.509      -0.037       0.074
B_avg_TOTAL_STR_att           -0.0036      0.003     -1.443      0.150      -0.009       0.001
B_avg_SIG_STR_landed          -0.0137      0.063     -0.218      0.828      -0.138       0.110
B_avg_opp_GROUND_att           0.0022      0.017      0.131      0.896      -0.031       0.035
R_avg_opp_KD                  -0.0313      0.125     -0.249      0.803      -0.278       0.216
R_avg_TOTAL_STR_att           -0.0005      0.003     -0.180      0.858      -0.006       0.005
B_avg_DISTANCE_att            -0.0027      0.005     -0.590      0.556      -0.012       0.006
B_avg_opp_TOTAL_STR_landed     0.0047      0.002      2.305      0.022       0.001       0.009
R_avg_opp_DISTANCE_att        -0.0036      0.003     -1.345      0.180      -0.009       0.002
B_avg_opp_GROUND_landed       -0.0067      0.025     -0.269      0.788      -0.055       0.042
R_avg_SIG_STR_landed           0.0105      0.032      0.331      0.741      -0.052       0.073
R_avg_opp_HEAD_landed         -0.0086      0.005     -1.799      0.073      -0.018       0.001
R_avg_DISTANCE_att             0.0079      0.005      1.430      0.154      -0.003       0.019
==============================================================================
Omnibus:                        5.164   Durbin-Watson:                   1.948
Prob(Omnibus):                  0.076   Jarque-Bera (JB):                3.642
Skew:                          -0.140   Prob(JB):                        0.162
Kurtosis:                       2.497   Cond. No.                     5.14e+03
============================

___
## Conclusão

   ___
## Referências

[Como usar a biblioteca Scikit-lear](https://scikit-learn.org/stable/modules/linear_model.html#logistic-regression)

[Como funciona o Random Forest](https://towardsdatascience.com/understanding-random-forest-58381e0602d2)

[Referencia do Random Forest Classifier](https://scikit-learn.org/stable/modules/generated/sklearn.ensemble.RandomForestClassifier.html)

[Como funciona regressão linear](https://www.saedsayad.com/logistic_regression.htm)

[Como usar a Regressão Linear](https://scikit-learn.org/stable/modules/generated/sklearn.linear_model.LinearRegression.html)



In [399]:
#criando um novo dataframe
lista_locB = ["B_fighter", "weight_class","B_Height_cms","B_Reach_cms","B_Weight_lbs","B_age"]
lista_locR = ["R_fighter", "weight_class","R_Height_cms","R_Reach_cms","R_Weight_lbs","R_age"]
data_red = data2.loc[: , lista_locR]
data_blue = data2.loc[: , lista_locB]

data_blue.columns = ["Fighter","Weight_class","Height_cms","Reach_cms","Weight_lbs","Age"]
data_red.columns = ["Fighter","Weight_class","Height_cms","Reach_cms","Weight_lbs","Age"]

data_red.head(2)

,Fighter,Weight_class,Height_cms,Reach_cms,Weight_lbs,Age
0,Henry Cejudo,Bantamweight,162.56,162.56,135.0,32.0
1,Valentina Shevchenko,Women's Flyweight,165.10,167.64,125.0,31.0


In [400]:
newdataR = pd.DataFrame()


namelist = []

for n in range(0,5143):
    if data_red.loc[n,:]["Fighter"] not in namelist:
        namelist.append(data_red.loc[n,:]["Fighter"])
        newdataR = pd.concat([newdataR,data_red.loc[n,:]], axis=1, join='outer')

newdataR = newdataR.transpose()

newdataR.head(2)

C:\ProgramData\Anaconda3\lib\site-packages\ipykernel_launcher.py:9: FutureWarning: Sorting because non-concatenation axis is not aligned. A future version
of pandas will change to not sort by default.

To accept the future behavior, pass 'sort=False'.

To retain the current behavior and silence the warning, pass 'sort=True'.

  if __name__ == '__main__':


,Age,Fighter,Height_cms,Reach_cms,Weight_class,Weight_lbs
0,32,Henry Cejudo,162.56,162.56,Bantamweight,135
1,31,Valentina Shevchenko,165.1,167.64,Women's Flyweight,125


In [401]:
newdataB = pd.DataFrame()

namelist = []

for n in range(0,5143):
    if data_blue.loc[n,:]["Fighter"] not in namelist:
        namelist.append(data_blue.loc[n,:]["Fighter"])
        newdataB = pd.concat([newdataB,data_blue.loc[n,:]], axis=1, join='outer')

newdataB = newdataB.transpose()

newdataB.head(2)

C:\ProgramData\Anaconda3\lib\site-packages\ipykernel_launcher.py:8: FutureWarning: Sorting because non-concatenation axis is not aligned. A future version
of pandas will change to not sort by default.

To accept the future behavior, pass 'sort=False'.

To retain the current behavior and silence the warning, pass 'sort=True'.

  


,Age,Fighter,Height_cms,Reach_cms,Weight_class,Weight_lbs
0,31,Marlon Moraes,167.64,170.18,Bantamweight,135
1,32,Jessica Eye,167.64,167.64,Women's Flyweight,125


In [402]:
newdata = pd.concat([newdataB, newdataR], axis=0, join='outer', ignore_index=False, keys=None,levels=None, names=None, verify_integrity=False, copy=True)
newdata = newdata.set_index("Fighter")

newdata.head(2)

,Age,Height_cms,Reach_cms,Weight_class,Weight_lbs
Fighter,,,,,
Marlon Moraes,31,167.64,170.18,Bantamweight,135
Jessica Eye,32,167.64,167.64,Women's Flyweight,125
